Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "05")
cd(ProjDir)

Read the milk data

In [3]:
wd = CSV.read(rel_path("..", "data", "milk.csv"), delim=';')
df = convert(DataFrame, wd);
dcc = filter(row -> !(row[:neocortex_perc] == "NA"), df)
dcc[:kcal_per_g] = convert(Vector{Float64}, dcc[:kcal_per_g])
dcc[:neocortex_perc] = parse.(Float64, dcc[:neocortex_perc])

17-element Array{Float64,1}:
 55.16
 64.54
 64.54
 67.64
 68.85
 58.85
 61.69
 60.32
 69.97
 70.41
 73.4 
 67.53
 71.26
 72.6 
 70.24
 76.3 
 75.49

Show first 5 rows

In [4]:
first(dcc, 5)

,clade,species,kcal_per_g,perc_fat,perc_protein,perc_lactose,mass,neocortex_perc
,String⍰,String⍰,Float64,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64
1,Strepsirrhine,Eulemur fulvus,0.49,16.6,15.42,67.98,1.95,55.16
2,New World Monkey,Alouatta seniculus,0.47,21.22,23.58,55.2,5.25,64.54
3,New World Monkey,A palliata,0.56,29.66,23.46,46.88,5.37,64.54
4,New World Monkey,Cebus apella,0.89,53.41,15.8,30.79,2.51,67.64
5,New World Monkey,S sciureus,0.92,50.58,22.33,27.09,0.68,68.85


Define the Stan language model

In [5]:
m5_5 = "
data{
    int N;
    vector[N] kcal_per_g;
    vector[N] neocortex_perc;
}
parameters{
    real a;
    real bn;
    real sigma;
}
model{
    vector[N] mu = a + bn * neocortex_perc;
    sigma ~ uniform( 0 , 1 );
    bn ~ normal( 0 , 1 );
    a ~ normal( 0 , 100 );
    kcal_per_g ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [6]:
stanmodel = Stanmodel(name="m5_5",
monitors = ["a", "bn", "sigma"],
 model=m5_5, output_format=:mcmcchains);
# Input data for cmdstan
m5_5_data = Dict("N" => size(dcc, 1),
  "kcal_per_g" => dcc[:kcal_per_g],
  "neocortex_perc" => dcc[:neocortex_perc]);
# Sample using cmdstan
rc, chn, cnames = stan(stanmodel, m5_5_data, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)


File /Users/rob/.julia/dev/StanModels/scripts/05/tmp/m5_5.stan will be updated.


Inference for Stan model: m5_5_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.35, 0.35, 0.12, 0.068) seconds, 0.88 seconds total
Sampling took (0.17, 0.18, 0.080, 0.056) seconds, 0.49 seconds total

                    Mean     MCSE  StdDev        5%       50%    95%    N_Eff  N_Eff/s    R_hat
lp__            -2.3e+04  1.6e+04   23297  -5.0e+04   1.1e+01     22  2.0e+00  4.1e+00  6.6e+02
accept_stat__    8.2e-01  8.7e-02    0.24   0.0e+00   9.1e-01   1.00  7.7e+00  1.6e+01  1.2e+00
stepsize__       2.9e-02  2.0e-02   0.028   5.0e-04   5.5e-02  0.059  2.0e+00  4.1e+00  2.6e+14
treedepth__      3.3e+00  9.4e-01     1.8   0.0e+00   3.0e+00    6.0  3.5e+00  7.2e+00  1.5e+00
n_leapfrog__     2.4e+01  1.2e+01      24   1.0e+00   1.4e+01     63  4.0e+00  8.1e+00  1.4e+00
divergent__      4.7e-01      nan    0.50   0.0e+00   0.0e+00 

Rethinking results

In [7]:
rethinking_results = "
        mean    sd   5.5%  94.5% n_eff  Rhat
a     -0.814 0.000 -0.815 -0.814     7 1.124
bn    -0.499 0.006 -0.508 -0.490     2 2.803
sigma  1.000 0.000  1.000  1.000    42 0.999
sigma  1.53 0.16  1.28  1.80  1121    1
"

"\n        mean    sd   5.5%  94.5% n_eff  Rhat\na     -0.814 0.000 -0.815 -0.814     7 1.124\nbn    -0.499 0.006 -0.508 -0.490     2 2.803\nsigma  1.000 0.000  1.000  1.000    42 0.999\nsigma  1.53 0.16  1.28  1.80  1121    1\n"

End of `05/5.5s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*